In [1]:
import proj1_helpers
import implementations
import numpy as np

In [2]:
print("Extracting dataset")
y_train_ref_source, X_train_ref_source, id1 = proj1_helpers.load_csv_data("train.csv")
#y_test_ref, X_test_ref, id2 = proj1_helpers.load_csv_data("test.csv")


Extracting dataset


In [45]:
i = 0
X_in = []
X_out = []
y_in = []
y_out=[]
for i in range(X_train_ref_source.shape[0]):
    x = X_train_ref_source[i]
    y = y_train_ref_source[i]
    if -999 in x:
        X_in.append(x)
        y_in.append(y)
    else:
        X_out.append(x)
        y_out.append(y)
        
X_in = np.array(X_in)
X_out = np.array(X_out)
y_in = np.array(y_in)
y_out = np.array(y_out)
    
print(X_in.shape,X_out.shape)

X_in_train_ref, X_in_test_ref, y_in_train_ref, y_in_test_ref = implementations.split_data(X_in, y_in, 1,23)
X_out_train_ref, X_out_test_ref, y_out_train_ref, y_out_test_ref = implementations.split_data(X_out, y_out, 1,23)

(181886, 30) (68114, 30)


In [46]:
def batch_ridge_in(lambda_, degree, batch_size):
    x = []
    xt = []
    for i in range(1,degree+1):
        x.append(np.power(X_in_train_ref,i))
        xt.append(np.power(X_in_test_ref,i))
        
    X_train = np.concatenate(x,axis = 1)
    y_train = y_in_train_ref
    X_test = np.concatenate(xt,axis = 1)
    y_test = y_in_test_ref
    
    #print("X_train shape : ", X_train.shape,"    X_test.shape : ", X_test.shape)
    #print("y_train shape : ", y_train.shape,"    y_test.shape : ", y_test.shape)
    
    num_features = X_train[0].shape[0]
    #print("number of features : ", num_features)

    #print("Splitting dataset into batch")
    X_batch = np.array_split(X_train, 1)
    y_batch = np.array_split(y_train, 1)

    #print(X_batch[0].shape)
    
    w = []
    i=0
    for y,batch in zip(y_batch,X_batch):
        w.append(implementations.ridge_regression(y, batch,lambda_))
    
    
    wp =  np.sum(np.array(w),axis=0)/(len(X_batch))
    
    
    

    y_pred_ridge_train = proj1_helpers.predict_labels(wp,X_train)
    y_pred_ridge_test = proj1_helpers.predict_labels(wp,X_test)
    s_tr = 0
    s_te = 0
    tot_tr = 0
    for i,y in enumerate(y_train):
        if y == y_pred_ridge_train[i]:
            s_tr += 1
        tot_tr += 1
    tot_te = 0
    for i,y in enumerate(y_test):
        if y == y_pred_ridge_test[i]:
            s_te += 1
        tot_te += 1

    print("Ridge precision on train : ",s_tr/tot_tr) #     Ridge precision on test : ",s_te/tot_te)
    
    return wp

In [47]:
wp_in = batch_ridge_in(300,6,32)

Ridge precision on train :  0.8036792276480873


In [50]:
def batch_ridge_out(lambda_, degree, batch_size):
    x = []
    xt = []
    for i in range(1,degree+1):
        x.append(np.power(X_out_train_ref,i))
        xt.append(np.power(X_out_test_ref,i))
        
    X_train = np.concatenate(x,axis = 1)
    y_train = y_out_train_ref
    X_test = np.concatenate(xt,axis = 1)
    y_test = y_out_test_ref
    
    #print("X_train shape : ", X_train.shape,"    X_test.shape : ", X_test.shape)
    #print("y_train shape : ", y_train.shape,"    y_test.shape : ", y_test.shape)
    
    num_features = X_train[0].shape[0]
    #print("number of features : ", num_features)

    #print("Splitting dataset into batch")
    X_batch = np.array_split(X_train, 1)
    y_batch = np.array_split(y_train, 1)

    #print(X_batch[0].shape)
    
    w = []
    i=0
    for y,batch in zip(y_batch,X_batch):
        w.append(implementations.ridge_regression(y, batch,lambda_))
    
    
    wp =  np.sum(np.array(w),axis=0)/(len(X_batch))
    
    
    

    y_pred_ridge_train = proj1_helpers.predict_labels(wp,X_train)
    y_pred_ridge_test = proj1_helpers.predict_labels(wp,X_test)
    s_tr = 0
    s_te = 0
    tot_tr = 0
    for i,y in enumerate(y_train):
        if y == y_pred_ridge_train[i]:
            s_tr += 1
        tot_tr += 1
    tot_te = 0
    for i,y in enumerate(y_test):
        if y == y_pred_ridge_test[i]:
            s_te += 1
        tot_te += 1

    print("Ridge precision on train : ",s_tr/tot_tr)#,"      Ridge precision on test : ",s_te/tot_te)
    
    return wp

In [51]:
wp_out = batch_ridge_out(300,7,32)


Ridge precision on train :  0.812681680711748


In [43]:
ytest, input_data_test, ids_test = proj1_helpers.load_csv_data("test.csv")
xsub = []
degree = 7
for i in range(1,degree+1):
    xsub.append(np.power(input_data_test,i))
inp = np.concatenate(xsub,axis = 1)



In [52]:
ypred = []
for i in range(inp.shape[0]):
    x = inp[i]
    if -999 in x:
        ypred.append(np.dot(wp_in,x[:180]))
    else:
        ypred.append(np.dot(wp_out,x))
ypred = np.array(ypred)
ypred[np.where(ypred <= 0)] = -1
ypred[np.where(ypred > 0)] = 1
proj1_helpers.create_csv_submission(ids_test, ypred, "pred4.csv")

In [42]:
s_te = 0
tot_te = 0
for i,y in enumerate(ytest):
    if y == ypred[i]:
        s_te += 1
    tot_te += 1

print("Ridge precision on train : ",s_te/tot_te)#     Ridge precision on test : ",s_te/tot_te)
    

Ridge precision on train :  0.807976


In [23]:
print(ypred)

[-1. -1. -1. ..., -1.  1. -1.]
